# Create the IPT model from scratch.

`-` This code requires **COMSOL 6.1** and **Python 3.11.4** or later versions.

`-` Each code cell should be executed *in order*.

`-` A simulation model will be automatically generated in the *saving path* if everything goes well. ^_^

## 1. Initialize COMSOL 3D model

In [1]:
import mph                                                                 # mph is the core module that used for the interaction between code and COMSOL
                                                                           # https://mph.readthedocs.io/en/1.2/index.html
import numpy as np
import copy
import time


path = 'E:\\Circular_Coil_IPT'                                             # Saving path of the generated COMSOL model

client = mph.start()                                                       # Start COMSOL client
model = client.create(path)                                                # Import temporal empty COMSOL file

components = model/'components'                       
components.create(True, name='comp1')                                      # Create model components
geometries = model/'geometries'
geometry = geometries.create(3, name='geom1')                              # Create 3d geometry space                       
model.java.component("comp1").geom("geom1").lengthUnit("mm")               # Change unit in geometry space  



parameters = model/'parameters'                                            # Input the parameters for IPT model
(parameters/'Parameters 1').rename('parameters')

model.parameter('f', '85[kHz]')
model.description('f', 'Operation frequency')
model.parameter('IL1', '30[A]')
model.description('IL1', 'Resonant current of primary side')
model.parameter('IL2', '0[A]')
model.description('IL2', 'Resonant current of secondary side')
model.parameter('d', '150[mm]')
model.description('d', 'Transfer distance')
model.parameter('l_coil_core', '5.6[mm]')
model.description('l_coil_core', 'Vertical distance between coil and core')
model.parameter('N1', '11')
model.description('N1', 'Number of turns on primary side')
model.parameter('N2', '11')
model.description('N2', 'Number of turns on secondary side')
model.parameter('r0_1', '2.1[mm]')
model.description('r0_1', 'Radius of primary litz wire')
model.parameter('R0_1', '158[mm]')
model.description('R0_1', 'Minor radius of primary coil')
model.parameter('R0_1', '158[mm]')
model.description('R0_1', 'Minor radius of primary coil')
model.parameter('ra_1', '6[mm]')
model.description('ra_1', 'Radial distance of primary coil')
model.parameter('r0_2', '2.1[mm]')
model.description('r0_2', 'Radius of primary litz wire')
model.parameter('R0_2', '158[mm]')
model.description('R0_2', 'Minor radius of primary coil')
model.parameter('R0_2', '158[mm]')
model.description('R0_2', 'Minor radius of primary coil')
model.parameter('ra_2', '6[mm]')
model.description('ra_2', 'Radial distance of primary coil')
model.parameter('PC95_weight', '50[mm]')
model.description('PC95_weight', 'Weight of PC95 core')
model.parameter('PC95_depth', '50[mm]')
model.description('PC95_depth', 'Depth of PC95 core')
model.parameter('PC95_height', '5[mm]')
model.description('PC95_height', 'Height of PC95 core')
model.parameter('Nano_weight', '50[mm]')
model.description('Nano_weight', 'Weight of nanocrystalline core')
model.parameter('Nano_depth', '50[mm]')
model.description('Nano_depth', 'Depth of nanocrystalline core')
model.parameter('Nano_height', '3[mm]')
model.description('Nano_height', 'Height of nanocrystalline core')

model.save(path)                                                           # The above operation will only take effect after saving

## 2. Establish Geometry

In [2]:
"""
In order to accelerate the calculation speed and increase the model's versatility, 
We employ the homogenized multiturn method to construct the circular coil structure.

"""

Cylinder1 = geometry.create('Cylinder', name='Cylinder1')                  # Primary side coil
Cylinder1.property("r", 'R0_1')
Cylinder1.property("h", '2*r0_1')
Cylinder1.property("r", "R0_1+ra_1*N1")
Cylinder1.property("pos", ["0", "0", "-d/2-r0_1"])
geometry.java.run("cyl1")

Cylinder2 = geometry.create('Cylinder', name='Cylinder2')                 
Cylinder2.property("r", 'R0_1')
Cylinder2.property("h", '2*r0_1')
Cylinder2.property("r", "R0_1")
Cylinder2.property("pos", ["0", "0", "-d/2-r0_1"])
geometry.java.run("cyl2")

Difference1 = geometry.create('Difference', name='Difference1')
Difference1.java.selection("input").set("cyl1")
Difference1.java.selection("input2").set("cyl2")
geometry.java.run("dif1")



Cylinder3 = geometry.create('Cylinder', name='Cylinder3')                 # Secondary side coil
Cylinder3.property("r", 'R0_2')
Cylinder3.property("h", '2*r0_2')
Cylinder3.property("r", "R0_2+ra_2*N2")
Cylinder3.property("pos", ["0", "0", "d/2-r0_2"])
geometry.java.run("cyl3")

Cylinder4 = geometry.create('Cylinder', name='Cylinder4')
Cylinder4.property("r", 'R0_2')
Cylinder4.property("h", '2*r0_2')
Cylinder4.property("r", "R0_2")
Cylinder4.property("pos", ["0", "0", "d/2-r0_2"])
geometry.java.run("cyl4")

Difference2 = geometry.create('Difference', name='Difference2')
Difference2.java.selection("input").set("cyl3")
Difference2.java.selection("input2").set("cyl4")
geometry.java.run("dif2")



"""
The core plate is composed of 81 small magnetic cores (length: 50mm and width: 50mm), arranged to form a 9x9 planar magnetic core matrix. 
The thickness varies based on the magnetic core material.

The default core material is MnZn ferrite PC95.

"""

num = 1                                                                  # Primary side core
core_matrix_row = 9
core_matrix_column = 9
for i in range(core_matrix_row):
    for j in range(core_matrix_column):
        core_b = geometry.create('Block', name='core'+str(num))
        core_b.property('size', ['PC95_weight', 'PC95_depth', 'PC95_height'])
        core_b.property('base', 'corner')
        core_b.property('pos', ['-(' + str(core_matrix_row) + '*PC95_weight/2-PC95_weight/2)' + '+' + str(i) + '*50' + '-25', 
                                '-(' + str(core_matrix_row) + '*PC95_weight/2-PC95_weight/2)' + '+' + str(j) + '*50' + '-25',
                                '-d/2-l_coil_core-PC95_height/2'])
        geometry.java.run("blk"+str(num))
        num = num + 1

core_b = geometry.create("ExplicitSelection", name="core b")             # sel1                  Default naming of ExplicitSelection 
num_b = 1                                                                # ExplicitSelection     Used for selecting domains in subsequent physics and meshes.
for i in range(core_matrix_row):
    for j in range(core_matrix_column):
        core_b.java.selection("selection").set("blk"+str(num_b), 1)
        num_b = num_b + 1


for i in range(core_matrix_row):                                         # Secondary side core
    for j in range(core_matrix_column):
        core_t = geometry.create('Block', name='core'+str(num))
        core_t.property('size', ['PC95_weight', 'PC95_depth', 'PC95_height'])
        core_t.property('base', 'corner')
        core_t.property('pos', ['-(' + str(core_matrix_row) + '*PC95_weight/2-PC95_weight/2)' + '+' + str(i) + '*50' + '-25', 
                                '-(' + str(core_matrix_row) + '*PC95_weight/2-PC95_weight/2)' + '+' + str(j) + '*50' + '-25', 
                                'd/2+l_coil_core-PC95_height/2'])
        geometry.java.run("blk"+str(num))
        num = num + 1
        
core_t = geometry.create("ExplicitSelection", name="core t")             # sel2
num_t = copy.deepcopy(num_b)
for i in range(core_matrix_row):
    for j in range(core_matrix_column):
        core_t.java.selection("selection").set("blk"+str(num_t), 1)
        num_t = num_t + 1

        

        
"""
For the homogenized multiturn coil model, a section is needed to set the resonant current in the coil.

"""        

Workplane1 = geometry.create('WorkPlane', name='Workplane1')  
Workplane1.property("unite", "on")
Workplane1.property("quickplane", "yz")
geometry.java.run('wp1')

PartitionDomains1 = geometry.create("PartitionDomains", name='PartitionDomains1')
PartitionDomains1.property("workplane", "wp1")
PartitionDomains1.java.selection("domain").set("dif1", 1)
PartitionDomains1.java.selection("domain").set("dif2", 1)
geometry.java.run('pard1')

coil_b = geometry.create("ExplicitSelection", name="coil b")            # sel3
coil_b.java.selection("selection").set("pard1(1)", 1, 2)

coil_t = geometry.create("ExplicitSelection", name="coil t")            # sel4
coil_t.java.selection("selection").set("pard1(2)", 1, 2)

coil = geometry.create("ExplicitSelection", name="coil")                # sel5
coil.java.selection("selection").set("pard1(1)", 1, 2)
coil.java.selection("selection").set("pard1(2)", 1, 2)

current_input_b = geometry.create("ExplicitSelection", name="current input b")  # sel6
current_input_b.java.selection("selection").init(2)
current_input_b.java.selection("selection").set("pard1(1)", 7)

current_input_t = geometry.create("ExplicitSelection", name="current input t")  # sel7
current_input_t.java.selection("selection").init(2);
current_input_t.java.selection("selection").set("pard1(2)", 7)




"""
This step is to draw the air domain. To optimize the calculation speed by reducing the number of air grids, 
this model uses the infinite element domain method to construct a spherical air domain.

'Infinite element' refers to a domain stretched along a specific coordinate axis, which approximately forms an infinite domain.

"""

air_domain = geometry.create("Sphere", name="air domain")                                     # Add a layer of spherical shell outside the spherical air domain for infinite element domain
air_domain.property("r", 500)
air_domain.property("layer", '50')
geometry.java.run('sph1')

air_domain_infinite = geometry.create("ExplicitSelection", name="air domain infinite")        # sel8
air_domain_infinite.java.selection("selection").set("sph1", 1, 2, 3, 4, 6, 7, 8, 9)           


core_all = geometry.create("ExplicitSelection", name="core_all")                              # sel9
for i in range(2*core_matrix_row*core_matrix_column):                                         # ExplicitSelection for core
    core_all.java.selection("selection").set("blk"+str(i+1), 1)

geometry.java.run('fin')

air_domain_small = geometry.create("ExplicitSelection", name="air domain small")              # sel10
air_domain_small.java.selection("selection").set("fin", 5)

air_domain_all = geometry.create("ExplicitSelection", name="air domain all")                  # sel11
air_domain_all.java.selection("selection").set("fin", 1, 2, 3, 4, 5, 98, 99, 102, 103)



model.save(path)     

## 3. Establish Physics

In [3]:
"""
The electromagnetic fields is calculated by magnetic fields module, 
while the external excitation of the coils is provided by the circuit module.

"""

physics = model/'physics'
mf = physics.create('InductionCurrents', geometry, name='Magnetic Fields')                  # Import magnetic fields
mf_core_b = mf.create("AmperesLaw", name="Core b")
mf_core_b.java.selection().named("geom1_sel1")
mf_core_b.property("materialType", "from_mat")
mf_core_b.property("ConstitutiveRelationBH", "RelativePermeability")                        # All the magnetic cores operate in the unsaturated region. 
                                                                                            # To reduce the complexity of the model, the constitutive relation B-H is implemented using relative permeability. 

mf_core_t = mf.create("AmperesLaw", name="Core t")
mf_core_t.java.selection().named("geom1_sel2")
mf_core_t.property("materialType", "from_mat")
mf_core_t.property("ConstitutiveRelationBH", "RelativePermeability")

mf_coil_b = mf.create("Coil", name="Coil b")
mf_coil_b.java.selection().named("geom1_sel3")
mf_coil_b.property("materialType", "from_mat")
mf_coil_b.property("ConductorModel", "Multi")                                               # Set homogenized multiturn method
mf_coil_b.property("CoilType", "Numeric")
mf_coil_b.property("CoilExcitation", "CircuitCurrent")                                      # Set external circuit excitation
mf_coil_b.property("N", "N1")
mf_coil_b.property("coilWindArea", "pi*r0_1*r0_1")
mf_coil_b.java.feature("ccc1").feature("ct1").selection().named("geom1_sel6")

mf_coil_t = mf.create("Coil", name="Coil t")
mf_coil_t.java.selection().named("geom1_sel4")
mf_coil_t.property("materialType", "from_mat")
mf_coil_t.property("ConductorModel", "Multi")
mf_coil_t.property("CoilType", "Numeric")
mf_coil_t.property("CoilExcitation", "CircuitCurrent")
mf_coil_t.property("N", "N2")
mf_coil_t.property("coilWindArea", "pi*r0_2*r0_2")
mf_coil_t.java.feature("ccc1").feature("ct1").selection().named("geom1_sel7")



cir = physics.create('Circuit', geometry, name='Circuit')                                   # Import circuit module
cir_coil_b = cir.create("ModelDeviceIV", name="Coil b")
cir_coil_b.property("Connections", ['0', '1'])
cir_coil_b.property("V_src", "root.comp1.mf.VCoil_1")

cir_source_b = cir.create("CurrentSourceCircuit", name="Source b")
cir_source_b.property("Connections", ['1', '0'])
cir_source_b.property("sourceType", "AC")
cir_source_b.property("value", "IL1")

cir_gnd_2 = cir.create("GroundNode", name="cir_gnd_2")
cir_gnd_2.property("Connections", ['2'])

cir_coil_t = cir.create("ModelDeviceIV", name="Coil t")
cir_coil_t.property("Connections", ['2', '3'])
cir_coil_t.property("V_src", "root.comp1.mf.VCoil_2")

cir_source_t = cir.create("CurrentSourceCircuit", name="Source t")
cir_source_t.property("Connections", ['3', '2'])
cir_source_t.property("sourceType", "AC")
cir_source_t.property("value", "IL2")



boundary = model.java.component("comp1").coordSystem().create("ie1", "InfiniteElement")     # Set infinite air domain
boundary.selection().named("geom1_sel8")
boundary.set("ScalingType", "Spherical")



model.save(path)     

## 4. Establish Materials

In [4]:
materials = model/'materials'

Nano_x_laminated = materials.create('Common', name='Nano_x_laminated')                      # X-laminated NRC 50mm*50mm*3mm
#Nano_x_laminated.java.selection().named("geom1_sel9")                                      # mat1
Nano_x_laminated.java.propertyGroup("def").set("relpermeability", ['4.545', '25740', '25740'])
Nano_x_laminated.java.propertyGroup("def").set("electricconductivity", ['34.31', '650000', '650000'])
Nano_x_laminated.java.propertyGroup("def").set("relpermittivity", '1')

Nano_y_laminated = materials.create('Common', name='Nano_y_laminated')                      # Y-laminated NRC 50mm*50mm*3mm
#Nano_y_laminated.java.selection().named("geom1_sel9")                                      # mat2
Nano_y_laminated.java.propertyGroup("def").set("relpermeability", ['25740', '4.545', '25740'])
Nano_y_laminated.java.propertyGroup("def").set("electricconductivity", ['650000', '34.31', '650000'])
Nano_y_laminated.java.propertyGroup("def").set("relpermittivity", '1')

PC95 = materials.create('Common', name='PC95')                                              # PC95 Ferrite core 50mm*50mm*5mm
PC95.java.selection().named("geom1_sel9")                                                   # mat3
PC95.java.propertyGroup("def").set("relpermeability", "3302.091549806689")
PC95.java.propertyGroup("def").set("electricconductivity", "0.167")
PC95.java.propertyGroup("def").set("relpermittivity", "1")

Air = materials.create('Common', name='Air')                                                # Air domain                                                 # mat1
Air.java.propertyGroup("def").set("relpermeability", "1")                                   # For material change operation
Air.java.propertyGroup("def").set("electricconductivity", "1")                              # mat4
Air.java.propertyGroup("def").set("relpermittivity", "1")

Air = materials.create('Common', name='Air domain')                                         # Air domain
Air.java.selection().named("geom1_sel11")                                                   # mat5
Air.java.propertyGroup("def").set("relpermeability", "1")
Air.java.propertyGroup("def").set("electricconductivity", "1")
Air.java.propertyGroup("def").set("relpermittivity", "1")

Copper = materials.create('Common', name='Copper')                                          # Coil(Copper)
Copper.java.selection().named("geom1_sel5")                                                 # mat6
Copper.java.propertyGroup("def").set("relpermeability", "1")
Copper.java.propertyGroup("def").set("electricconductivity", "5.998e7[S/m]")
Copper.java.propertyGroup("def").set("relpermittivity", "1")



model.save(path)     

## 5. Establish Mesh

In [5]:
meshes = model/'meshes'
mesh1 = meshes.create(geometry, name='mesh')

"""
Since infinite domains perform coordinate stretching in some way, 
the meshes needs to match the stretching direction. 

In 3D, the sweeping method should be used to generate infinite element domain's meshes.

"""

mesh_air_infinite = mesh1.create("Sweep", name='mesh_air_infinite') 
mesh_air_infinite.java.selection().named("geom1_sel8")
Distribution1 = mesh_air_infinite.create("Distribution", name='Distribution1')
Size1 = mesh_air_infinite.create("Size", name='Size1')
Size1.property("hauto", '5')


"""
In the remaining area, due to the irregular geometric shape of the internal air domain, a free tetrahedral mesh is used.

In addition, an adaptive coarsing mesh strategy is set up in the internal air domain to reduce the number of meshes.

"""

mesh_interior = mesh1.create("FreeTet", name='mesh_interior') 
mesh_interior.java.selection().remaining()
Size2 = mesh_interior.create("Size", name='Size2')
Size2.java.selection().named("geom1_sel5")
Size2.property("hauto", '5')
Size3 = mesh_interior.create("Size", name='Size3')
Size3.java.selection().named("geom1_sel9")
Size3.property("hauto", '5')
Size4 = mesh_interior.create("Size", name='Size4')
Size4.java.selection().named("geom1_sel10")
Size4.property("hauto", '6')
Size4.property("table", 'coarseadaptation')
mesh1.java.run()

model.save(path)     

## 6. Establish Solver

In [6]:
"""
In some cases in frequency domain analysis, the physical field is set correctly, but the stiffness matrix would still become pathological, 
such as geometries with very large aspect ratios or steep changes in material physical properties. 

For this reason, the solution method of the model should be set to direct solver (usually iterative by default) 
and transferred to the direct nodes in the solver sequence, set the error estimate to 'No'.

"""


studies = model/'studies'
solutions = model/'solutions'

study = studies.create(name='solver1')
step1 = study.create("CoilCurrentCalculation", name='Coil Geometry Analysis')
step2 = study.create("Frequency", name='Frequency Domain')
step2.property("punit", "kHz")
step2.property("plist", "f")



model.java.sol().create("sol1")
model.java.sol("sol1").study("std1")
model.java.study("std1").feature("ccc").set("notlistsolnum", '1')
model.java.study("std1").feature("ccc").set("notsolnum", "auto")
model.java.study("std1").feature("ccc").set("listsolnum", '1')
model.java.study("std1").feature("ccc").set("solnum", "auto")
model.java.study("std1").feature("freq").set("notlistsolnum", '1')
model.java.study("std1").feature("freq").set("notsolnum", "auto")
model.java.study("std1").feature("freq").set("listsolnum", '1')
model.java.study("std1").feature("freq").set("solnum", "auto")

model.java.sol("sol1").create("st1", "StudyStep")
model.java.sol("sol1").feature("st1").set("study", "std1")
model.java.sol("sol1").feature("st1").set("studystep", "ccc")
model.java.sol("sol1").create("v1", "Variables")
model.java.sol("sol1").feature("v1").set("control", "ccc")
model.java.sol("sol1").create("s1", "Stationary")
model.java.sol("sol1").feature("s1").create("se1", "Segregated")
model.java.sol("sol1").feature("s1").feature("se1").feature().remove("ssDef")
model.java.sol("sol1").feature("s1").feature("se1").create("ss1", "SegregatedStep")
model.java.sol("sol1").feature("s1").feature("se1").feature("ss1").set("segvar", ["comp1_mf_coil1_ccc1_s", "comp1_mf_coil2_ccc1_s"])
model.java.sol("sol1").feature("s1").feature("se1").feature("ss1").set("linsolver", "dDef")
model.java.sol("sol1").feature("s1").feature("se1").create("ss2", "SegregatedStep")
model.java.sol("sol1").feature("s1").feature("se1").feature("ss2").set("segvar", ["comp1_mf_coil1_ccc1_p", "comp1_mf_coil1_ccc1_lm", "comp1_mf_coil2_ccc1_p", "comp1_mf_coil2_ccc1_lm"])
model.java.sol("sol1").feature("s1").feature("se1").feature("ss2").set("linsolver", "dDef")
model.java.sol("sol1").feature("s1").feature("se1").set("segterm", "itertol")
model.java.sol("sol1").feature("s1").feature("se1").set("segiter", '6')
model.java.sol("sol1").feature("s1").feature().remove("fcDef")
model.java.sol("sol1").create("su1", "StoreSolution")



model.java.sol("sol1").create("st2", "StudyStep")
model.java.sol("sol1").feature("st2").set("study", "std1")
model.java.sol("sol1").feature("st2").set("studystep", "freq")
model.java.sol("sol1").create("v2", "Variables")
model.java.sol("sol1").feature("v2").set("initmethod", "sol")
model.java.sol("sol1").feature("v2").set("initsol", "sol2")
model.java.sol("sol1").feature("v2").set("notsolmethod", "sol")
model.java.sol("sol1").feature("v2").set("notsol", "sol2")
model.java.sol("sol1").feature("v2").set("control", "freq")
model.java.sol("sol1").create("s2", "Stationary")
model.java.sol("sol1").feature("s2").create("p1", "Parametric")
model.java.sol("sol1").feature("s2").feature().remove("pDef")
model.java.sol("sol1").feature("s2").feature("p1").set("pname", ["freq"])
model.java.sol("sol1").feature("s2").feature("p1").set("plistarr", ["f"])
model.java.sol("sol1").feature("s2").feature("p1").set("punit", ["kHz"])
model.java.sol("sol1").feature("s2").feature("p1").set("pcontinuationmode", "no")
model.java.sol("sol1").feature("s2").feature("p1").set("preusesol", "auto")
model.java.sol("sol1").feature("s2").feature("p1").set("pdistrib", "off")
model.java.sol("sol1").feature("s2").feature("p1").set("plot", "off")
model.java.sol("sol1").feature("s2").feature("p1").set("plotgroup", "Default")
model.java.sol("sol1").feature("s2").feature("p1").set("probesel", "all")
#model.java.sol("sol1").feature("s2").feature("p1").set("probes", [])
model.java.sol("sol1").feature("s2").feature("p1").set("control", "freq")
model.java.sol("sol1").feature("s2").set("linpmethod", "sol")
model.java.sol("sol1").feature("s2").set("linpsol", "zero")
model.java.sol("sol1").feature("s2").set("control", "freq")
model.java.sol("sol1").feature("s2").create("iDef", "Iterative")
model.java.sol("sol1").feature("s2").create("fc1", "FullyCoupled")
model.java.sol("sol1").feature("s2").create("i1", "Iterative")
model.java.sol("sol1").feature("s2").feature("i1").set("linsolver", "bicgstab")
model.java.sol("sol1").feature("s2").feature("i1").set("nlinnormuse", 'on')
model.java.sol("sol1").feature("s2").feature("i1").set("prefuntype", "right")
model.java.sol("sol1").feature("s2").feature("i1").create("mg1", "Multigrid")
model.java.sol("sol1").feature("s2").feature("i1").feature("mg1").feature("pr").create("sv1", "SORVector")
model.java.sol("sol1").feature("s2").feature("i1").feature("mg1").feature("pr").feature("sv1").set("prefun", "sorvec")
model.java.sol("sol1").feature("s2").feature("i1").feature("mg1").feature("pr").feature("sv1").set("iter", '2')
model.java.sol("sol1").feature("s2").feature("i1").feature("mg1").feature("pr").feature("sv1").set("relax", '1')
model.java.sol("sol1").feature("s2").feature("i1").feature("mg1").feature("pr").feature("sv1").set("sorvecdof", ["comp1_A"])
model.java.sol("sol1").feature("s2").feature("i1").feature("mg1").feature("po").create("sv1", "SORVector")
model.java.sol("sol1").feature("s2").feature("i1").feature("mg1").feature("po").feature("sv1").set("prefun", "soruvec")
model.java.sol("sol1").feature("s2").feature("i1").feature("mg1").feature("po").feature("sv1").set("iter", '2')
model.java.sol("sol1").feature("s2").feature("i1").feature("mg1").feature("po").feature("sv1").set("relax", '1')
model.java.sol("sol1").feature("s2").feature("i1").feature("mg1").feature("po").feature("sv1").set("sorvecdof", ["comp1_A"])
model.java.sol("sol1").feature("s2").feature("fc1").set("linsolver", "i1")
model.java.sol("sol1").feature("s2").feature().remove("fcDef")
model.java.sol("sol1").feature("s2").feature().remove("iDef")

model.java.sol("sol1").feature("s2").feature("fc1").set("linsolver", "dDef")
model.java.sol("sol1").feature("s2").feature("dDef").set("errorchk", "off")


model.save(path)

In [7]:
mph.tree(model)                               # show the model tree

Circular_Coil_IPT
├─ parameters
│  └─ parameters
├─ functions
│  ├─ Pulse source
│  ├─ Smoothed ramp
│  ├─ Maximum limit
│  ├─ Safe exponential
│  ├─ Step
│  └─ Step
├─ components
│  └─ comp1
├─ geometries
│  └─ geom1
│     ├─ Cylinder1
│     ├─ Cylinder2
│     ├─ Difference1
│     ├─ Cylinder3
│     ├─ Cylinder4
│     ├─ Difference2
│     ├─ core1
│     ├─ core2
│     ├─ core3
│     ├─ core4
│     ├─ core5
│     ├─ core6
│     ├─ core7
│     ├─ core8
│     ├─ core9
│     ├─ core10
│     ├─ core11
│     ├─ core12
│     ├─ core13
│     ├─ core14
│     ├─ core15
│     ├─ core16
│     ├─ core17
│     ├─ core18
│     ├─ core19
│     ├─ core20
│     ├─ core21
│     ├─ core22
│     ├─ core23
│     ├─ core24
│     ├─ core25
│     ├─ core26
│     ├─ core27
│     ├─ core28
│     ├─ core29
│     ├─ core30
│     ├─ core31
│     ├─ core32
│     ├─ core33
│     ├─ core34
│     ├─ core35
│     ├─ core36
│     ├─ core37
│     ├─ core38
│     ├─ core39
│     ├─ core40
│     ├─ core41
│     ├─ core42
│

In [8]:
client.remove(model)                          # Release the model file from the code                         
client.clear()                                # Close the client

# Calculation

In [9]:
import mph                                                                 #
import numpy as np
import copy
import time
"""
To modify the magnetic core material, it is necessary to index the geometry based on the default number given by COMSOL. 
Therefore, it is necessary to first create a 9 * 9 index matrix for the magnetic core matrix.

"""

bottom_row_b = np.array([6, 26, 44, 62, 80, 104, 122, 140, 158])    
bottom_row_t = np.array([7, 27, 45, 63, 81, 105, 123, 141, 159]) 

core_b_num = np.zeros((9, 9))
core_t_num = np.zeros((9, 9))

for i in range(8, -1, -1):
    core_b_num[i, :] = bottom_row_b + (8 - i) * 2
    core_t_num[i, :] = bottom_row_t + (8 - i) * 2

core_b_num = core_b_num.astype(int)
core_t_num = core_t_num.astype(int)



"""
Magnetic core layout matrix: The replacement of the magnetic core material can be achieved by modifying the symbol at certain position in core_layout_matrix. 
The materials provided in this code are as follows:
 
Type                                   Default name     Symbol           Size  
NRC107B (X-laminated)                     mat1             0         50mm*50mm*3mm
NRC107B (Y-laminated)                     mat2             1         50mm*50mm*3mm
PC95 Ferrite                              mat3             2         50mm*50mm*5mm
Air Core:                                 mat4             3             none



It is worth noting that mat3 and mat4 are the same material, but due to its anisotropy, 
the orientation will affect the magnetic field distribution.

To distinguish its arrangement direction, it is treated as two different materials in the code.
"""

core_layout_matrix = np.array([[2, 2, 2, 2, 2, 2, 2, 2, 2],
                               [2, 2, 2, 2, 2, 2, 2, 2, 2],
                               [2, 2, 2, 1, 0, 0, 2, 2, 2],
                               [2, 2, 1, 1, 0, 0, 0, 2, 2],
                               [2, 2, 1, 1, 0, 1, 1, 2, 2],
                               [2, 2, 0, 0, 0, 1, 1, 2, 2],
                               [2, 2, 2, 0, 0, 1, 2, 2, 2],
                               [2, 2, 2, 2, 2, 2, 2, 2, 2],
                               [2, 2, 2, 2, 2, 2, 2, 2, 2]])                      # Change the material of core matrix at there




def get_inductance(core_layout_matrix, path):
    
    client = mph.start()   
    model = client.load(path)      
    
    core_layout_matrix = np.reshape(core_layout_matrix, (9,9))
    core_materials_b = core_layout_matrix
    core_materials_t = np.copy(np.rot90(core_layout_matrix, k=-3))
    core_materials_b = np.round(core_materials_b).astype(np.int32) 
    core_materials_t = np.round(core_materials_t).astype(np.int32)
    
    mat1 = []
    mat2 = []
    mat3 = []
    mat4 = []
    for i in range(9):
        for j in range(9):
            eval('mat'+str(core_materials_b[8-i, j]+1)).append(core_b_num[8-i, j])
            eval('mat'+str(core_materials_t[8-i, j]+1)).append(core_t_num[8-i, j])
            blk_b = "blk"+str(1 + j + (9 - (8 - i + 1)) * 9)
            blk_t = "blk"+str(81+1+j+(9-(i+1))*9)  
            pos_nano = [str((j-4)*50 - 25), str((i-4)*50 - 25), "d/2+l_coil_core-PC95_height/2+(PC95_height-Nano_height)"]
            pos_fe = [str((j-4)*50 - 25), str((i-4)*50 - 25), "d/2+l_coil_core-PC95_height/2"]
            
            if core_materials_b[j, 8-i] in [0,1]:
                model.java.component("comp1").geom("geom1").feature(blk_b).set("size", ['Nano_weight', 'Nano_depth', 'Nano_height'])
            if core_materials_t[j, 8-i] in [0,1]:
                model.java.component("comp1").geom("geom1").feature(blk_t).set("size", ['Nano_weight', 'Nano_depth', 'Nano_height'])
                model.java.component("comp1").geom("geom1").feature(blk_t).set("pos", pos_nano) 
            if core_materials_b[j, 8-i] in [2,3]:
                model.java.component("comp1").geom("geom1").feature(blk_b).set("size", ['PC95_weight', 'PC95_depth', 'PC95_height'])
            if core_materials_t[j, 8-i] in [2,3]:
                model.java.component("comp1").geom("geom1").feature(blk_t).set("size", ['PC95_weight', 'PC95_depth', 'PC95_height'])
                model.java.component("comp1").geom("geom1").feature(blk_t).set("pos", pos_fe)
                
    
    (model/'materials'/'Nano_x_laminated').select(mat1)
    (model/'materials'/'Nano_y_laminated').select(mat2)
    (model/'materials'/'PC95').select(mat3)
    (model/'materials'/'Air').select(mat4)        
    
    
    model.save(path)
    
    start = time.time()
    model.java.sol("sol1").runAll()
    end = time.time()
    
    L = (model.evaluate('mf.LCoil_1','H'))*1000000 
    M = (model.evaluate('abs(mf.PhiCoil_2/mf.ICoil_1)','H'))*1000000 
    k_coff = (model.evaluate('abs(mf.PhiCoil_2/mf.ICoil_1)/sqrt(mf.LCoil_1*mf.LCoil_1)','1'))*1
    
    client.remove(model)                                            
    client.clear()
    
    print("Self-inductance:", L, "μH")
    print("Mutual-inductance:", M, "μH")
    print("Coupling coefficient:", k_coff)
    print("Computation time:", end-start, "s")
    
path = 'E:\\Circular_Coil_IPT'  
get_inductance(core_layout_matrix, path)

Self-inductance: 133.57637298390654 μH
Mutual-inductance: 37.650035266231235 μH
Coupling coefficient: 0.28186148811487316
Computation time: 68.51681113243103 s
